In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

In [ ]:
AMZN_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/AMZN_predictions.csv", index_col=0)


In [ ]:
AMZN_predictions.head()

In [ ]:
AMZN_predictions.Date = pd.to_datetime(AMZN_predictions.Date, format="%Y-%m-%d")

In [ ]:
AMZN_predictions['Predictions'] = AMZN_predictions['Predictions'].shift(-1)


In [ ]:
AMZN_predictions = AMZN_predictions.dropna()


In [ ]:
AMZN_predictions = AMZN_predictions[::-1]

In [ ]:
AMZN_predictions.head(10)

In [ ]:
AMZN_monthly = pd.read_csv("/Users/fahad/project_repo/data/external/AMZN_monthly.csv", index_col =0)

In [ ]:
AMZN_monthly.columns = ['Open','High','Low','Close','Volume']

In [ ]:
AMZN_predictions = AMZN_predictions.set_index("Date")

In [ ]:
AMZN_monthly.head()

In [ ]:
AMZN_final_df = pd.merge(AMZN_predictions, AMZN_monthly, right_index=True, left_index=True)

In [ ]:
AMZN_final_df.index = pd.to_datetime(AMZN_final_df.index, format ="%Y-%m-%d")

In [ ]:
equity_dict = dict()

In [ ]:
class BH(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.set_trailing_sl(4)
        self.start = self.data.index[0]
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
#         print (type(self.start))
        if self.data.index[-1] > self.start and (position.is_long!=True):
        
            self.buy()
        else:
            pass

In [ ]:
bt = Backtest(AMZN_final_df, BH, cash=10000, commission=.01)
bt.run()



In [ ]:
bt.plot(superimpose=False, plot_volume=False)

In [ ]:
df = pd.DataFrame.from_dict([equity_dict])

In [ ]:
df = df.T

In [ ]:
df.columns = ['Equity']

In [ ]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/BH/AMZN_BH_Equity.csv")

In [ ]:
df